# SMU Logo bounding box detection
Design an algorithm to identify all images of SMU from a set of random images. A bounding box should be drawn around the SMU logo whenever it appears in an image.

When an SMU logo occurs in an image, a bounding box should be drawn around it. Logo identification will be evaluated by its F1 score.

### Setup
Install dependencies
```bash
pip install -r requirements.txt

```

In [1]:
%pip install -r requirements.txt

  Obtaining dependency information for ultralytics==8.0.196 from https://files.pythonhosted.org/packages/a8/97/d6d2592ba629ab41e18aaba14a4d75a5b56c40398a1c0b9e5979317798d0/ultralytics-8.0.196-py3-none-any.whl.metadata
  Obtaining dependency information for roboflow from https://files.pythonhosted.org/packages/32/98/117f2db9078fb16f9a7e1084ac904750f1bc877d5a85eff4f6c2d7f70a28/roboflow-1.1.21-py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python>=4.6.0 from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for torch>=1.8.0 from https://files.pythonhosted.org/packages/59/1f/4975d1ab3ed2244053876321ef65bc02935daed67da76c6e7d65900772a3/torch-2.2.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchvision>=0.9.0 from https://files.pythonhosted.org/packages/e7/45/419aa0b37254f1fd62b45bb63836066c

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.10.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Import packages

In [1]:
from ultralytics import YOLO
from roboflow import Roboflow
from PIL import Image
from dotenv import load_dotenv
import os

Load environment variables using dotenv
- `API_KEY` - API Key for Roboflow dataset
- `DATA_VERSION` - Version of dataset used
- `PATH_TO_DATA` - Absolute file path to the "SMU-Logo-Detection-1" folder. See .env.example for example

In [2]:
# Project settings
load_dotenv()
api_key = os.getenv('API_KEY')
data_version = os.getenv('DATA_VERSION')
path_to_data = f'{os.getenv("PATH_TO_PROJECT")}/tree-type-detection-{data_version}'

Load dataset into workspace

In [3]:
rf = Roboflow(api_key=api_key)
project = rf.workspace("tree-dataset-iftyz").project("tree-type-detection-9rfxy")
dataset = project.version(data_version).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


Update test, train and val values in data.yaml
This fixes a file not found bug

In [5]:
!python update_path.py

### [Training](https://docs.ultralytics.com/modes/train/#train-settings)
Use a trained model, saved in `/saved_models`, or train a model.
A pre-trained [model](https://github.com/ultralytics/ultralytics?tab=readme-ov-file) `yolov8n.pt` from Ultralytics should be used. Tune hyper-params such as learning rate and epochs.

If there is file not found error, check the `FILE_TO_PATH` env variable, and make sure that you've updated the `data.yml` to the absolute file path.

In [ ]:
# # Train model (Laptop)
# model = YOLO('yolov8n.pt') # pre-trained yolov8 nano
# results = model.train(data=f'{path_to_data}/data.yaml', epochs=10)

# # Train model (SMU GPU)
# model = YOLO('yolov8n.pt')
# results = model.train(data=f'{path_to_data}/data.yaml', epochs=10, batch=-1, device=0, workers=4)

In [3]:
# Use saved model
model = YOLO('./saved_models/v4_epoch10.pt')

After training, the model can be found in `/runs`. Save the `best.pt` model in the `/saved_models` folder.

### [Validation](https://docs.ultralytics.com/modes/val/#usage-examples)
Val mode is used for validating a YOLOv8 model after it has been trained. In this mode, the model is evaluated on a validation set to measure its accuracy and generalization performance. This mode can be used to tune the hyperparameters of the model to improve its performance.

In [4]:
model.val(data=f'{path_to_data}/data.yaml', conf=0.25, iou=0.6)
'''
1. Epochs: Vary from 7, 15 etc
2. Find a way to output the predicted labels into a folder. Compare the predicted labels and actual labels to check the metrics
3. Re-annotate if have time (those with very small logos)


4. annotate the rest of the images
5. after everything is annotated, augmentation and final training
'''

Ultralytics YOLOv8.0.196  Python-3.10.6 torch-2.2.1+cpu CPU (AMD Ryzen 9 5900HX with Radeon Graphics)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\valid\labels... 576 images, 463 backgrounds, 0 corrupt: 100%|██████████| 576/576 [00:00<00:00, 2399.99it/s]
val: New cache created: C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\valid\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:48<00:00,  1.35s/it]
                   all        576        129      0.939      0.721      0.846      0.674
Speed: 1.5ms preprocess, 75.9ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs\detect\val


'\n1. Epochs: Vary from 7, 15 etc\n2. Find a way to output the predicted labels into a folder. Compare the predicted labels and actual labels to check the metrics\n3. Re-annotate if have time (those with very small logos)\n\n\n4. annotate the rest of the images\n5. after everything is annotated, augmentation and final training\n'

### [Test](https://docs.ultralytics.com/modes/predict/#why-use-ultralytics-yolo-for-inference)
Predict bounding boxes on test set

In [46]:
def convert_bbox_format(prediction):
  # Extract information from the prediction object
  image_path = prediction.path.split("\\")[-1]  # Extract image name
  box = prediction.boxes.xyxy.tolist()[0]  # Get top-left and bottom-right coordinates

  # Calculate width and height
  width = box[2] - box[0]
  height = box[3] - box[1]

  return [image_path, box[0], box[1], width, height]

In [5]:
# Predict
prediction = model.predict(source=f"{path_to_data}/test/images", conf=0.25, iou=0.6)


image 1/288 C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\test\images\IMG_8053_JPG.rf.2d2db88e7f26cda9e376dbafe963a9d9.jpg: 640x640 (no detections), 82.0ms
image 2/288 C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\test\images\IMG_8067_JPG.rf.730df81b052e107a9fce34b09b875eb6.jpg: 640x640 (no detections), 67.0ms
image 3/288 C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\test\images\IMG_8075_JPG.rf.0280008f3f6e1d5c33599c629843eeed.jpg: 640x640 (no detections), 83.0ms
image 4/288 C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\test\images\IMG_8090_JPG.rf.fdacda6e11509d87bb5fc07923be1c36.jpg: 640x640 (no detections), 69.0ms
image 5/288 C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\test\images\IMG_8165_JPG.rf.fc2fd99a3964df0741ace08abf6c88fe.jpg: 640x640 (no detections), 68.0ms
image 6/288 C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\test\images\IMG_8

Save images to `/predictions` folder if bounding box exists.

In [61]:
images = {}

if not os.path.exists(f'{path_to_data}/../predictions'):
    os.makedirs(f'{path_to_data}/../predictions')

for i in range(0, len(prediction)):
    if prediction[i].boxes.shape[0] >= 1:
        im_rgb = Image.fromarray(prediction[i].plot(line_width=1)[..., ::-1])
        images[i] = im_rgb  # RGB-order PIL image
        predicted_box = convert_bbox_format(prediction[i])

        # Write labels
        with open(f'{path_to_data}/../predictions/obj_est.txt', 'a') as f:
            f.write(', '.join(map(str, predicted_box)) + '\n')
        # Save images
        im_rgb.save(fp=f'{path_to_data}/../predictions/{predicted_box[0]}_labelled.jpg')

len(images)

49